# 주제 선정 : 경기도 반려동물 빅데이터를 이용한 사업부지 선정

In [13]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# csv가 깨져서 utf로 변환하여 다시 저장해서 꺼내 쓰는 방법을 사용했습니다.

In [14]:
# 자료 원본 불러오기
pop_data = pd.read_csv('주민등록인구집계현황.csv', encoding='euc-kr')
pet_data = pd.read_csv('반려동물등록현황.csv', encoding='euc-kr')
store_data = pd.read_csv('반려동물판매업체현황.csv',encoding='cp949')

# utf-8로 저장
pop_data.to_csv('utf-주민등록인구집계현황.csv', encoding='utf-8', index=None)
pet_data.to_csv('utf-반려동물등록현황.csv', encoding='utf-8', index=None)
store_data.to_csv('utf-반려동물판매업체현황.csv', encoding='utf-8', index=None)

# 인구 자료 다시 불러오기
pop_data = pd.read_csv('utf-주민등록인구집계현황.csv')
pet_data = pd.read_csv('utf-반려동물등록현황.csv')
store_data = pd.read_csv('utf-반려동물판매업체현황.csv')

# 동물 복지 현황

경기도에는 반려동물이 어디에 얼마나 살고 있을까?

In [15]:
pet_data

,시군명,읍면동명,등록동물수(마리),(등록주체)시군구등록,(등록주체)대행업체등록,(등록주체)기타,(RFID종류)내장형,(RFID종류)외장형,(RFID종류)인식표,등록품종수,등록소유자수,동물소유자당등록동물수,해당동의등록대행업체수,데이터기준일자
0,가평군,가평읍,941,NaN,NaN,NaN,596,294,51,NaN,85.0,NaN,3.0,2022-06-14
1,가평군,북면,289,NaN,NaN,NaN,176,93,20,NaN,185.0,NaN,0.0,2022-06-14
2,가평군,상면,399,NaN,NaN,NaN,217,153,29,NaN,243.0,NaN,0.0,2022-06-14
3,가평군,설악면,1111,NaN,NaN,NaN,446,612,53,NaN,625.0,NaN,1.0,2022-06-14
4,가평군,조종면,416,NaN,NaN,NaN,218,161,37,NaN,274.0,NaN,1.0,2022-06-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,화성시,청계동,2046,90.0,1956.0,0.0,1052,879,115,66.0,1679.0,1.22,4.0,2022-05-31
725,화성시,팔탄면,753,87.0,665.0,1.0,345,305,103,62.0,412.0,1.83,1.0,2022-05-31
726,화성시,향남읍,5180,314.0,4863.0,3.0,2110,2584,486,103.0,3719.0,1.39,6.0,2022-05-31
727,화성시,활초동,15,0.0,15.0,0.0,4,11,0,7.0,11.0,1.36,0.0,2022-05-31


### 여러 데이터 중 중요한 자료는 지역명들과 동물의 수, 기타 몇가지라고 판단했습니다.

In [16]:
pet = pet_data.loc[:,['시군명','읍면동명','등록동물수(마리)','등록소유자수','해당동의등록대행업체수']]
pet

,시군명,읍면동명,등록동물수(마리),등록소유자수,해당동의등록대행업체수
0,가평군,가평읍,941,85.0,3.0
1,가평군,북면,289,185.0,0.0
2,가평군,상면,399,243.0,0.0
3,가평군,설악면,1111,625.0,1.0
4,가평군,조종면,416,274.0,1.0
...,...,...,...,...,...
724,화성시,청계동,2046,1679.0,4.0
725,화성시,팔탄면,753,412.0,1.0
726,화성시,향남읍,5180,3719.0,6.0
727,화성시,활초동,15,11.0,0.0


#### 또한 시군명이 읍면동 단위로 내려가면서 시군 단위에서의 동물의 수가 의미가 있다고 판단했습니다.

- 시군명 자료만 리스트로 만들어서 지역별 동물의 수를 붙여서 데이터프레임화하였습니다.

In [17]:
town_num = pet.drop_duplicates(['시군명'], keep='first', inplace=False, ignore_index=True)
town = list(town_num['시군명'])
print(town)

['가평군', '고양시', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시', '수원시', '시흥시', '안산시', '안성시', '안양시', '양주시', '양평군', '여주시', '연천군', '오산시', '용인시', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시']


In [18]:
ani_list = []
for i in town:
    ani_sum = pet[pet['시군명'].str.contains(i)]['등록동물수(마리)'].sum()
#     ani_list.append([)
#     ani_list.append(ani_sum)
    ani_list.append([i, ani_sum])
print(ani_list)

[['가평군', 4017], ['고양시', 73477], ['과천시', 2974], ['광명시', 20161], ['광주시', 29042], ['구리시', 12381], ['군포시', 15480], ['김포시', 7912], ['남양주시', 42957], ['동두천시', 6998], ['부천시', 113892], ['성남시', 55728], ['수원시', 65330], ['시흥시', 30539], ['안산시', 86158], ['안성시', 19950], ['안양시', 35644], ['양주시', 54805], ['양평군', 13291], ['여주시', 5252], ['연천군', 2020], ['오산시', 14471], ['용인시', 50227], ['의왕시', 6590], ['의정부시', 58997], ['이천시', 13147], ['파주시', 24057], ['평택시', 60468], ['포천시', 11168], ['하남시', 17634], ['화성시', 48264]]


In [19]:
# 자료 정제
pet_df = pd.DataFrame(ani_list)
pet_df.columns = ['행정구역명', '총 동물수']
pet_df

,행정구역명,총 동물수
0,가평군,4017
1,고양시,73477
2,과천시,2974
3,광명시,20161
4,광주시,29042
5,구리시,12381
6,군포시,15480
7,김포시,7912
8,남양주시,42957
9,동두천시,6998


# 그렇다면 이 동물들은 얼마나 많은 사람과 살고 있을까?

In [20]:
pop_data

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,9,도,경기도,13574353,1054380,1327378,1753988,1880750,2286448,...,646493,828547,904145,1122988,1131303,864393,435119,248574,43544,1373
1,2022,9,시군,경기도 가평군,62168,3047,4471,6009,5406,7722,...,2175,2520,2449,3575,5177,6376,3551,2379,395,15
2,2022,9,읍면동,경기도 가평군 가평읍,19532,1062,1677,2125,1804,2649,...,784,959,845,1245,1590,1792,1013,732,117,7
3,2022,9,읍면동,경기도 가평군 북면,3821,126,177,224,231,368,...,90,82,99,164,309,507,288,206,39,1
4,2022,9,읍면동,경기도 가평군 상면,5688,153,244,512,389,560,...,111,155,171,245,543,718,384,240,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107481,2008,1,읍면동,경기도 화성시 팔탄면,9900,869,869,1694,1671,1735,...,407,606,596,717,579,455,373,120,23,0
107482,2008,1,읍면동,경기도 화성시 향남읍,21100,2828,2288,3151,4316,3415,...,1136,1444,1835,1467,1107,797,622,180,34,4
107483,2008,1,읍면동,경기도 화성시 향남읍,21100,2828,2288,3151,4316,3415,...,1136,1444,1835,1467,1107,797,622,180,34,4
107484,2008,1,읍면동,경기도 화성시 화산동,23684,3727,2793,3515,5424,3869,...,1296,1699,2528,1737,1079,686,419,115,19,0


### 방대한 자료에서 필요한 자료는 최신 인구수라고 판단 2022년의 9월 자료 중 행정구역 구분명이 시군인 자료들만 
### 추렸습니다.

In [21]:
popu = pop_data[(pop_data['연도']==2022) & (pop_data['월']==9) & (pop_data['행정구역구분명']=='시군')]
popu

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
1,2022,9,시군,경기도 가평군,62168,3047,4471,6009,5406,7722,...,2175,2520,2449,3575,5177,6376,3551,2379,395,15
8,2022,9,시군,경기도 고양시,1079277,76769,101359,144797,142834,175981,...,49590,70882,71693,90624,100944,67927,35157,23140,3937,114
56,2022,9,시군,경기도 과천시,78301,7252,7477,10114,11678,11865,...,3668,4833,5884,6037,6823,5019,2314,1375,323,9
63,2022,9,시군,경기도 광명시,290756,21241,29236,36023,39235,49324,...,14175,17320,19193,25074,24582,20764,9739,5302,899,26
83,2022,9,시군,경기도 광주시,388893,32763,34308,44419,55022,68394,...,16730,20789,26395,32423,32169,26827,12073,5872,1070,42
97,2022,9,시군,경기도 구리시,191011,13565,17140,25407,24761,31133,...,8355,12350,12264,15557,17728,13257,6438,3334,501,13
106,2022,9,시군,경기도 군포시,267493,18143,25052,36251,37090,42096,...,12094,17369,17597,21308,24232,18749,8356,5258,945,39
119,2022,9,시군,경기도 김포시,485609,52226,50418,51534,73073,93045,...,24751,25096,37234,44817,34193,27972,14407,7329,1329,35
134,2022,9,시군,경기도 남양주시,734642,61907,78547,85553,89129,132532,...,38089,41173,45023,66282,60538,47333,25339,13121,2299,94
153,2022,9,시군,경기도 동두천시,93260,5643,8397,10848,9843,13662,...,4162,4986,4686,6626,7799,7291,4859,3094,448,15


### 자료의 가독성을 위해 필요한 자료인 연도, 행정구역명, 총 인구수 자료로 추립니다.
### 앞에서 확보한 총 동물 수 자료와 지역명을 맞추기 위해 컬럼 자료값에서 '경기도'를 제외합니다.

In [22]:
population_df = popu.loc[:, ['연도','행정구역명','총 인구수']]
population_df['행정구역명'] = ['가평군', '고양시', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', 
'동두천시', '부천시', '성남시', '수원시', '시흥시', '안산시', '안성시', '안양시', '양주시', '양평군', '여주시', '연천군', 
'오산시', '용인시', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시']
population_df

,연도,행정구역명,총 인구수
1,2022,가평군,62168
8,2022,고양시,1079277
56,2022,과천시,78301
63,2022,광명시,290756
83,2022,광주시,388893
97,2022,구리시,191011
106,2022,군포시,267493
119,2022,김포시,485609
134,2022,남양주시,734642
153,2022,동두천시,93260


# 자료 병합 : 어디에 얼마나 많은 사람과 함께 많은 동물들이 살고 있는지 살펴봅시다.

In [23]:
p_a_sum = pd.merge(pet_df, population_df, how="outer")
p_a_sum

,행정구역명,총 동물수,연도,총 인구수
0,가평군,4017,2022,62168
1,고양시,73477,2022,1079277
2,과천시,2974,2022,78301
3,광명시,20161,2022,290756
4,광주시,29042,2022,388893
5,구리시,12381,2022,191011
6,군포시,15480,2022,267493
7,김포시,7912,2022,485609
8,남양주시,42957,2022,734642
9,동두천시,6998,2022,93260


In [24]:
# 자료 정렬
stat = p_a_sum.sort_values(by=['총 인구수','총 동물수'],ascending=False)
stat

,행정구역명,총 동물수,연도,총 인구수
12,수원시,65330,2022,1188234
1,고양시,73477,2022,1079277
22,용인시,50227,2022,1075877
11,성남시,55728,2022,928267
30,화성시,48264,2022,892038
10,부천시,113892,2022,801503
8,남양주시,42957,2022,734642
14,안산시,86158,2022,650708
27,평택시,60468,2022,569369
16,안양시,35644,2022,549724


In [25]:
# 가독성을 위해 컬럼 순서 변경
stat[['연도','행정구역명','총 인구수','총 동물수']]

,연도,행정구역명,총 인구수,총 동물수
12,2022,수원시,1188234,65330
1,2022,고양시,1079277,73477
22,2022,용인시,1075877,50227
11,2022,성남시,928267,55728
30,2022,화성시,892038,48264
10,2022,부천시,801503,113892
8,2022,남양주시,734642,42957
14,2022,안산시,650708,86158
27,2022,평택시,569369,60468
16,2022,안양시,549724,35644


#### 특이점 : 부천시의 경우 인구는 80만밖에 안되지만 동물 수가 11만마리가 넘는다. 

# 도시별 반려동물 관련 산업의 분포 파악

In [28]:
# 인구 자료 
pop_data = pd.read_csv('data1/주민등록인구집계현황.csv', encoding='euc-kr')
pop_data

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,9,도,경기도,13574353,1054380,1327378,1753988,1880750,2286448,...,646493,828547,904145,1122988,1131303,864393,435119,248574,43544,1373
1,2022,9,시군,경기도 가평군,62168,3047,4471,6009,5406,7722,...,2175,2520,2449,3575,5177,6376,3551,2379,395,15
2,2022,9,읍면동,경기도 가평군 가평읍,19532,1062,1677,2125,1804,2649,...,784,959,845,1245,1590,1792,1013,732,117,7
3,2022,9,읍면동,경기도 가평군 북면,3821,126,177,224,231,368,...,90,82,99,164,309,507,288,206,39,1
4,2022,9,읍면동,경기도 가평군 상면,5688,153,244,512,389,560,...,111,155,171,245,543,718,384,240,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107481,2008,1,읍면동,경기도 화성시 팔탄면,9900,869,869,1694,1671,1735,...,407,606,596,717,579,455,373,120,23,0
107482,2008,1,읍면동,경기도 화성시 향남읍,21100,2828,2288,3151,4316,3415,...,1136,1444,1835,1467,1107,797,622,180,34,4
107483,2008,1,읍면동,경기도 화성시 향남읍,21100,2828,2288,3151,4316,3415,...,1136,1444,1835,1467,1107,797,622,180,34,4
107484,2008,1,읍면동,경기도 화성시 화산동,23684,3727,2793,3515,5424,3869,...,1296,1699,2528,1737,1079,686,419,115,19,0


In [29]:
# 2022년 9월 자료
over_pop = pop_data[(pop_data['연도']==2022) & (pop_data['월']==9) & (pop_data['행정구역구분명']=='시군')]
over_pop

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
1,2022,9,시군,경기도 가평군,62168,3047,4471,6009,5406,7722,...,2175,2520,2449,3575,5177,6376,3551,2379,395,15
8,2022,9,시군,경기도 고양시,1079277,76769,101359,144797,142834,175981,...,49590,70882,71693,90624,100944,67927,35157,23140,3937,114
56,2022,9,시군,경기도 과천시,78301,7252,7477,10114,11678,11865,...,3668,4833,5884,6037,6823,5019,2314,1375,323,9
63,2022,9,시군,경기도 광명시,290756,21241,29236,36023,39235,49324,...,14175,17320,19193,25074,24582,20764,9739,5302,899,26
83,2022,9,시군,경기도 광주시,388893,32763,34308,44419,55022,68394,...,16730,20789,26395,32423,32169,26827,12073,5872,1070,42
97,2022,9,시군,경기도 구리시,191011,13565,17140,25407,24761,31133,...,8355,12350,12264,15557,17728,13257,6438,3334,501,13
106,2022,9,시군,경기도 군포시,267493,18143,25052,36251,37090,42096,...,12094,17369,17597,21308,24232,18749,8356,5258,945,39
119,2022,9,시군,경기도 김포시,485609,52226,50418,51534,73073,93045,...,24751,25096,37234,44817,34193,27972,14407,7329,1329,35
134,2022,9,시군,경기도 남양주시,734642,61907,78547,85553,89129,132532,...,38089,41173,45023,66282,60538,47333,25339,13121,2299,94
153,2022,9,시군,경기도 동두천시,93260,5643,8397,10848,9843,13662,...,4162,4986,4686,6626,7799,7291,4859,3094,448,15


In [30]:
pop = over_pop.loc[:, ['연도','행정구역명','총 인구수']]
pop['행정구역명'] = ['가평군', '고양시', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', 
'동두천시', '부천시', '성남시', '수원시', '시흥시', '안산시', '안성시', '안양시', '양주시', '양평군', '여주시', '연천군', 
'오산시', '용인시', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시']
pop

,연도,행정구역명,총 인구수
1,2022,가평군,62168
8,2022,고양시,1079277
56,2022,과천시,78301
63,2022,광명시,290756
83,2022,광주시,388893
97,2022,구리시,191011
106,2022,군포시,267493
119,2022,김포시,485609
134,2022,남양주시,734642
153,2022,동두천시,93260


In [31]:
# 동물 통계

In [32]:
# 데이터 기준 일자 2022-06-14
pet_data = pd.read_csv('data1/반려동물등록현황.csv', encoding='euc-kr')
pet_data

,시군명,읍면동명,등록동물수(마리),(등록주체)시군구등록,(등록주체)대행업체등록,(등록주체)기타,(RFID종류)내장형,(RFID종류)외장형,(RFID종류)인식표,등록품종수,등록소유자수,동물소유자당등록동물수,해당동의등록대행업체수,데이터기준일자
0,가평군,가평읍,941,NaN,NaN,NaN,596,294,51,NaN,85.0,NaN,3.0,2022-06-14
1,가평군,북면,289,NaN,NaN,NaN,176,93,20,NaN,185.0,NaN,0.0,2022-06-14
2,가평군,상면,399,NaN,NaN,NaN,217,153,29,NaN,243.0,NaN,0.0,2022-06-14
3,가평군,설악면,1111,NaN,NaN,NaN,446,612,53,NaN,625.0,NaN,1.0,2022-06-14
4,가평군,조종면,416,NaN,NaN,NaN,218,161,37,NaN,274.0,NaN,1.0,2022-06-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,화성시,청계동,2046,90.0,1956.0,0.0,1052,879,115,66.0,1679.0,1.22,4.0,2022-05-31
725,화성시,팔탄면,753,87.0,665.0,1.0,345,305,103,62.0,412.0,1.83,1.0,2022-05-31
726,화성시,향남읍,5180,314.0,4863.0,3.0,2110,2584,486,103.0,3719.0,1.39,6.0,2022-05-31
727,화성시,활초동,15,0.0,15.0,0.0,4,11,0,7.0,11.0,1.36,0.0,2022-05-31


In [33]:
pet = pet_data.loc[:,['시군명','읍면동명','등록동물수(마리)','등록소유자수','해당동의등록대행업체수']]
pet

,시군명,읍면동명,등록동물수(마리),등록소유자수,해당동의등록대행업체수
0,가평군,가평읍,941,85.0,3.0
1,가평군,북면,289,185.0,0.0
2,가평군,상면,399,243.0,0.0
3,가평군,설악면,1111,625.0,1.0
4,가평군,조종면,416,274.0,1.0
...,...,...,...,...,...
724,화성시,청계동,2046,1679.0,4.0
725,화성시,팔탄면,753,412.0,1.0
726,화성시,향남읍,5180,3719.0,6.0
727,화성시,활초동,15,11.0,0.0


In [34]:
town_num = pet.drop_duplicates(['시군명'], keep='first', inplace=False, ignore_index=True)
town = list(town_num['시군명'])
town
print(len(town))

31


In [35]:
ani_list = []
for i in town:
    ani_sum = pet[pet['시군명'].str.contains(i)]['등록동물수(마리)'].sum()
#     ani_list.append([)
#     ani_list.append(ani_sum)
    ani_list.append([i, ani_sum])
print(ani_list)

[['가평군', 4017], ['고양시', 73477], ['과천시', 2974], ['광명시', 20161], ['광주시', 29042], ['구리시', 12381], ['군포시', 15480], ['김포시', 7912], ['남양주시', 42957], ['동두천시', 6998], ['부천시', 113892], ['성남시', 55728], ['수원시', 65330], ['시흥시', 30539], ['안산시', 86158], ['안성시', 19950], ['안양시', 35644], ['양주시', 54805], ['양평군', 13291], ['여주시', 5252], ['연천군', 2020], ['오산시', 14471], ['용인시', 50227], ['의왕시', 6590], ['의정부시', 58997], ['이천시', 13147], ['파주시', 24057], ['평택시', 60468], ['포천시', 11168], ['하남시', 17634], ['화성시', 48264]]


In [36]:
ani_df = pd.DataFrame(ani_list)
ani_df.columns = ['행정구역명', '총 동물수']
ani_df

,행정구역명,총 동물수
0,가평군,4017
1,고양시,73477
2,과천시,2974
3,광명시,20161
4,광주시,29042
5,구리시,12381
6,군포시,15480
7,김포시,7912
8,남양주시,42957
9,동두천시,6998


In [37]:
# 자료 합치기

In [38]:
stat1 = pd.merge(pop, ani_df, how="outer")
stat1

,연도,행정구역명,총 인구수,총 동물수
0,2022,가평군,62168,4017
1,2022,고양시,1079277,73477
2,2022,과천시,78301,2974
3,2022,광명시,290756,20161
4,2022,광주시,388893,29042
5,2022,구리시,191011,12381
6,2022,군포시,267493,15480
7,2022,김포시,485609,7912
8,2022,남양주시,734642,42957
9,2022,동두천시,93260,6998


In [39]:
stat1.sort_values(by=['총 인구수','총 동물수'],ascending=False)

,연도,행정구역명,총 인구수,총 동물수
12,2022,수원시,1188234,65330
1,2022,고양시,1079277,73477
22,2022,용인시,1075877,50227
11,2022,성남시,928267,55728
30,2022,화성시,892038,48264
10,2022,부천시,801503,113892
8,2022,남양주시,734642,42957
14,2022,안산시,650708,86158
27,2022,평택시,569369,60468
16,2022,안양시,549724,35644


In [40]:
# 반려동물 관련 업체

In [41]:
pet_store = pd.read_csv('data1/반려동물판매업체현황.csv',encoding='cp949')
pet_store

,시군명,사업장명,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,폐업일자,소재지시설전화번호,소재지면적정보,도로명우편번호,...,소재지지번주소,소재지우편번호,WGS84위도,WGS84경도,업태구분명정보,X좌표값,Y좌표값,축산업무구분명,축산고유번호,권리주체일련번호
0,가평군,예쁜강아지,20080909,NaN,4,말소,20220526.0,NaN,NaN,NaN,...,경기도 가평군 가평읍 읍내리 ***-*번지,NaN,NaN,NaN,NaN,245212.733716,480846.133947,동물관련영업,NaN,000
1,가평군,자이언트 펫,20160517,NaN,4,말소,20220526.0,031-585-7003,0.0,12457.0,...,경기도 가평군 청평면 대성리 ***-**번지,12457.0,NaN,NaN,NaN,233842.021227,466192.802959,동물관련영업,NaN,000
2,가평군,메이플켄넬,20170227,NaN,4,말소,NaN,585-0588,0.0,12416.0,...,경기도 가평군 가평읍 대곡리 ***-*,NaN,NaN,NaN,NaN,243247.301370,480064.513852,동물관련영업,0.0,000
3,가평군,리얼펫독,20150120,NaN,4,말소,20220526.0,NaN,0.0,12467.0,...,경기도 가평군 설악면 신천리 ***-*,NaN,NaN,NaN,NaN,243200.199051,463336.290376,동물관련영업,NaN,000
4,가평군,강아지나라,20100311,NaN,4,말소,20220526.0,NaN,NaN,NaN,...,경기도 가평군 청평면 청평리 ***-**번지,NaN,NaN,NaN,NaN,236894.589288,470644.366611,동물관련영업,NaN,000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14449,화성시,금은이네농장,20211119,NaN,2,폐업,20220728.0,NaN,0.0,18562.0,...,경기도 화성시 우정읍 운평리 ***,18562.0,NaN,NaN,NaN,180335.240868,397423.594307,동물관련영업,0.0,000
14450,화성시,냥이와 양이,20170530,NaN,1,휴업,NaN,NaN,0.0,18556.0,...,경기도 화성시 서신면 홍법리 ***-*,18556.0,NaN,NaN,NaN,175386.850094,407462.012505,동물관련영업,0.0,000
14451,NaN,학교종이 댕댕댕(24거3849),20200909,NaN,4,말소,20220526.0,NaN,0.0,31560.0,...,충청남도 아산시 법곡동 *-**,31560.0,NaN,NaN,NaN,201030.437190,362598.297324,동물관련영업,NaN,000
14452,NaN,쁘띠살롱,20220126,NaN,4,말소,20220526.0,NaN,0.0,21548.0,...,인천광역시 남동구 구월동 **-**,NaN,NaN,NaN,NaN,174682.069911,439615.145968,동물관련영업,0.0,000


In [42]:
store_num = pet_store[(pet_store['영업상태명']) =='정상'].loc[:,['시군명', '사업장명', '소재지지번주소']]
store_num

,시군명,사업장명,소재지지번주소
56,가평군,러브캣,경기도 가평군 상면 봉수리 **-*
57,가평군,가평가족동물학교,경기도 가평군 북면 백둔리 **
58,가평군,행복한고양이,경기도 가평군 가평읍 대곡리 ***-*
59,가평군,굿독,NaN
60,가평군,유로애견농원,경기도 가평군 설악면 신천리 ***
...,...,...,...
14431,화성시,샤르망,경기도 화성시 서신면 용두리 ***-*
14432,화성시,오스카코리아켄넬,경기도 화성시 서신면 홍법리 ***-*
14433,화성시,Steel Sprit,경기도 화성시 우정읍 멱우리 ***
14434,화성시,BLUE&CO,경기도 화성시 장안면 장안리 ***-*


In [43]:
store = store_num.drop_duplicates(['시군명'], keep='first', inplace=False, ignore_index=True)
shop = list(store['시군명'])
shop

['가평군',
 '고양시',
 '과천시',
 '광주시',
 '구리시',
 '군포시',
 '김포시',
 '남양주시',
 '동두천시',
 '부천시',
 '성남시',
 '수원시',
 '시흥시',
 '안산시',
 '안성시',
 '안양시',
 '양주시',
 '양평군',
 '여주시',
 '연천군',
 '오산시',
 '용인시',
 '의왕시',
 '의정부시',
 '이천시',
 '파주시',
 '평택시',
 '포천시',
 '하남시',
 '화성시']

In [44]:
shop_df = pd.DataFrame(shop)
shop_df.columns = ['행정구역명']
shop_df

,행정구역명
0,가평군
1,고양시
2,과천시
3,광주시
4,구리시
5,군포시
6,김포시
7,남양주시
8,동두천시
9,부천시


In [45]:
stat = pd.merge(stat1, shop_df, how="outer")
stat

,연도,행정구역명,총 인구수,총 동물수
0,2022,가평군,62168,4017
1,2022,고양시,1079277,73477
2,2022,과천시,78301,2974
3,2022,광명시,290756,20161
4,2022,광주시,388893,29042
5,2022,구리시,191011,12381
6,2022,군포시,267493,15480
7,2022,김포시,485609,7912
8,2022,남양주시,734642,42957
9,2022,동두천시,93260,6998


In [46]:
stat.to_csv('근도씨 정리파일.csv', encoding='utf-8')